[View in Colaboratory](https://colab.research.google.com/github/mogamin/chainer-examples/blob/master/chainer_PotatoChips_Classification_SSD2.ipynb)

In [2]:
![ ! -e ./data ] && curl -L "https://rebrand.ly/dllab2018-hackathon-cv" -o data.tar.gz && gzip -d -c data.tar.gz | tar xf -

# chainer,chainercv,cudaののセットアップ
!apt-get install -y -qq libcusparse8.0 libnvrtc8.0 libnvtoolsext1
!ln -snf /usr/lib/x86_64-linux-gnu/libnvrtc-builtins.so.8.0 /usr/lib/x86_64-linux-gnu/libnvrtc-builtins.so
!pip install cupy-cuda80==4.3.0 
!pip install chainer==4.3.0
!pip install chainercv

In [4]:
# GPU環境の確認
import chainer
print('GPU availability:', chainer.cuda.available)
print('cuDNN availablility:', chainer.cuda.cudnn_enabled)

# クリーニング
!rm -rf result/

GPU availability: True
cuDNN availablility: True


In [0]:
# 乱数表の固定化
import random
import numpy as np

RANDOM_SEED = 0
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
if chainer.cuda.available:
    chainer.cuda.cupy.random.seed(RANDOM_SEED)

# 実行時間の生成
import datetime
now = (datetime.datetime.now() + datetime.timedelta(hours=+9)).strftime('%Y%m%d-%H%M%S')

In [13]:
#!curl -sc /tmp/cookie "https://drive.google.com/uc?export=download&id=1DXBavZ6yst1Udhqld816vScDhk-MrtoB" > /dev/null
#!awk '/_warning_/ {print $NF}' /tmp/cookie
#!curl -Lb /tmp/cookie "https://drive.google.com/uc?export=download&confirm=Wm07&id=1DXBavZ6yst1Udhqld816vScDhk-MrtoB" -o model_potato_20180816-134135_gpu.npz
#!ls -l
#!mkdir -p bbox-images

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   388    0   388    0     0    388      0 --:--:-- --:--:-- --:--:--  4459
100  913M    0  913M    0     0   130M      0 --:--:--  0:00:07 --:--:--  137M
total 4467480
drwxrwxr-x 5 1008 1008       4096 Jun 19 13:19 data
lrwxrwxrwx 1 root root          8 Aug 15 20:56 datalab -> /content
-rw-r--r-- 1 root root 3616797934 Aug 20 05:47 data.tar.gz
-rw-r--r-- 1 root root  957880472 Aug 20 07:04 model_potato_20180816-134135_gpu.npz
drwxr-xr-x 2 root root       4096 Aug 15 20:56 sample_data


In [0]:
import copy
import numpy as np

import chainer
from chainer.datasets import ConcatenatedDataset
from chainer.datasets import TransformDataset
from chainer.optimizer_hooks import WeightDecay
from chainer import serializers
from chainer import training
from chainer.training import extensions
from chainer.training import triggers

from chainercv.datasets import voc_bbox_label_names
from chainercv.datasets import VOCBboxDataset
from chainercv.extensions import DetectionVOCEvaluator
from chainercv.links.model.ssd import GradientScaling
from chainercv.links.model.ssd import multibox_loss
from chainercv.links import SSD300
from chainercv.links import SSD512
from chainercv import transforms

from chainercv.links.model.ssd import random_crop_with_bbox_constraints
from chainercv.links.model.ssd import random_distort
from chainercv.links.model.ssd import resize_with_random_interpolation

In [0]:
class MultiboxTrainChain(chainer.Chain):

    def __init__(self, model, alpha=1, k=3):
        super(MultiboxTrainChain, self).__init__()
        with self.init_scope():
            self.model = model
        self.alpha = alpha
        self.k = k

    def __call__(self, imgs, gt_mb_locs, gt_mb_labels):
        mb_locs, mb_confs = self.model(imgs)
        loc_loss, conf_loss = multibox_loss(mb_locs, mb_confs, gt_mb_locs, gt_mb_labels, self.k)
        loss = loc_loss * self.alpha + conf_loss

        chainer.reporter.report(
            {'loss': loss, 'loss/loc': loc_loss, 'loss/conf': conf_loss},
            self)

        return loss

In [0]:
! rm -f bbox-images/*

from chainercv.transforms import resize
from chainercv.utils.image import write_image

class Transform(object):

    def __init__(self, coder, size, mean):
        # to send cpu, make a copy
        self.coder = copy.copy(coder)
        self.coder.to_cpu()

        self.size = size
        self.mean = mean
        self.count = 0

    def __call__(self, in_data):
        img, bbox, label = in_data
        #write_image(img, './bbox-images/'+str(self.count)+'_0.png')
        img = resize(img, (224, 224))

        # 1. Color augmentation
        img = random_distort(img)
        #write_image(img, './bbox-images/'+str(self.count)+'_1.png')

        # 2. Random expansion
        if np.random.randint(2):
            img, param = transforms.random_expand(img, fill=self.mean, return_param=True)
            bbox = transforms.translate_bbox(bbox, y_offset=param['y_offset'], x_offset=param['x_offset'])
        #write_image(img, './bbox-images/'+str(self.count)+'_2.png')

        # 3. Random cropping
        img, param = random_crop_with_bbox_constraints(img, bbox, return_param=True)
        bbox, param = transforms.crop_bbox(
            bbox, y_slice=param['y_slice'], x_slice=param['x_slice'],
            allow_outside_center=False, return_param=True)
        label = label[param['index']]
        #write_image(img, './bbox-images/'+str(self.count)+'_3.png')

        # 4. Resizing with random interpolatation
        _, H, W = img.shape
        img = resize_with_random_interpolation(img, (self.size, self.size))
        bbox = transforms.resize_bbox(bbox, (H, W), (self.size, self.size))
        #write_image(img, './bbox-images/'+str(self.count)+'_4.png')

        # 5. Random horizontal flipping
        img, params = transforms.random_flip(img, x_random=True, return_param=True)
        bbox = transforms.flip_bbox(bbox, (self.size, self.size), x_flip=params['x_flip'])
        #write_image(img, './bbox-images/'+str(self.count)+'_5.png')

        # Preparation for SSD network
        #img -= self.mean
        mb_loc, mb_label = self.coder.encode(bbox, label)
        #write_image(img, './bbox-images/'+str(self.count)+'_6.png')

        self.count += 1
        return img, mb_loc, mb_label

In [0]:
voc_bbox_label_names = (
    'ususio-BIGBAG','ususio-LBIG','ususio-NORMAL',
    'consome-NORMAL','consome-LBIG','consome-BIGBAG',
    'k-soy-BIGBAG','k-soy-LBAG','k-soy-NORMAL')

model = SSD300(n_fg_class=len(voc_bbox_label_names), pretrained_model='./model_potato_20180816-134135_gpu.npz')
#model = SSD300(n_fg_class=len(voc_bbox_label_names), pretrained_model='imagenet')
#model = SSD300(n_fg_class=20, pretrained_model='imagenet')

model.use_preset('evaluate')
train_chain = MultiboxTrainChain(model)
chainer.cuda.get_device_from_id(0).use()
model.to_gpu()

In [0]:
import numpy as np
import numpy
import os
from chainercv.chainer_experimental.datasets.sliceable import GetterDataset
from chainercv.utils import generate_random_bbox
import six

from PIL import Image
def _read_image_as_array(path, dtype):
    f = Image.open(path)
    try:
        image = numpy.asarray(f, dtype=dtype)
    finally:
        # Only pillow >= 3.0 has 'close' method
        if hasattr(f, 'close'):
            f.close()
    return image


def _postprocess_image(image):
    if image.ndim == 2:
        # image is greyscale
        image = image[..., None]
    return image.transpose(2, 0, 1)
  
  
class MyBboxDataset(GetterDataset):
  def __init__(self, pairs, root='.', dtype=None, label_dtype=numpy.int32):
    super(MyBboxDataset, self).__init__()
    
    # register getter methods
    self.add_getter('img', self.get_image)
    #self.add_getter(('bbox', 'label'), self.get_annotation)
    self.add_getter(('bbox', 'label', 'difficult'), self.get_annotation)
    
    if isinstance(pairs, six.string_types):
        pairs_path = pairs
        with open(pairs_path) as pairs_file:
            pairs = []
            for i, line in enumerate(pairs_file):
                pair = line.strip().split()
                if len(pair) != 2:
                    raise ValueError(
                        'invalid format at line {} in file {}'.format(
                            i, pairs_path))
                pairs.append((pair[0], int(pair[1])))
    self._pairs = pairs
    self._root = root
    self._dtype = np.float32
    self._label_dtype = label_dtype    
    self.keys = ('img', 'bbox', 'label')
    self.bbox = np.array([[ 50.0 , 50.0 , 170.0 , 170.0 ]])
    ##generate_random_bbox(1, (224, 224), 180, 190)

  def __len__(self):
    return len(self._pairs)
  
  def get_image(self, i):
    path, int_label = self._pairs[i]
    full_path = os.path.join(self._root, path)
    image = _read_image_as_array(full_path, self._dtype)
    return _postprocess_image(image)
      
  def get_annotation(self, i):
    #print("i=",i)
    #print(type(self.bbox))
    #print(self.bbox.shape)

    path, int_label = self._pairs[i]
    label = numpy.array([int_label], dtype=self._label_dtype)
    #label = int_label
    #print("label=",label)
        
    # generate dummy annotations
    #bbox = generate_random_bbox(1, (224, 224), 180, 190)
    return self.bbox, label

In [19]:
if 1:
  BATCHSIZE=32
  train = MyBboxDataset('data/train/train_labels.txt', 'data/train/images/')
  train = TransformDataset(train,  Transform(model.coder, model.insize, model.mean))
  train_iter = chainer.iterators.MultiprocessIterator(train, BATCHSIZE)
  

  #test = LabeledImageDataset('data/valid/valid_labels.txt', root='data/valid/images/')
  #test = TransformDataset(test, ImageResizeTransform())
  test = VOCBboxDataset(
      year='2007', split='test',
      use_difficult=True, return_difficult=True)

  test_iter = chainer.iterators.SerialIterator(test, BATCHSIZE, repeat=False, shuffle=False)
  
if 0:
  train = TransformDataset(
      ConcatenatedDataset(
          VOCBboxDataset(year='2007', split='trainval'),
          VOCBboxDataset(year='2012', split='trainval')
      ),
      Transform(model.coder, model.insize, model.mean))

  train_iter = chainer.iterators.MultiprocessIterator(train, 32)

  test = VOCBboxDataset(year='2007', split='test', use_difficult=True, return_difficult=True)
  test_iter = chainer.iterators.SerialIterator(test, 32, repeat=False, shuffle=False)

  from chainercv.visualizations import vis_bbox
  import matplotlib.pyplot as plt

  #img, bbox, label = train[0]
  #print(bbox.shape)
  #print(label.shape)
  #vis_bbox(img, bbox, label, label_names=voc_bbox_label_names)
  #plt.show()

From: http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtest_06-Nov-2007.tar
To: /content/.chainer/dataset/_dl_cache/c320d1dc0cd031efc99e256bf21d57a6
  %   Total    Recv       Speed  Time left
100  430MiB  430MiB   4332KiB/s    0:00:00

In [0]:
# initial lr is set to 1e-3 by ExponentialShift

#optimizer = chainer.optimizers.MomentumSGD()
#optimizer.setup(train_chain)
#for param in train_chain.params():
#    if param.name == 'b':
#        param.update_rule.add_hook(GradientScaling(2))
#    else:
#        param.update_rule.add_hook(WeightDecay(0.0005))
        
optimizer = chainer.optimizers.Adam()
optimizer.setup(model)
  
updater = training.updaters.StandardUpdater(train_iter, optimizer, device=0) 

In [21]:
trainer = training.Trainer(updater, (10, 'epoch'), 'result/ssd')
#trainer.extend(extensions.ExponentialShift('lr', 0.1, init=1e-4), trigger=(1,'epoch'))

#trainer.extend(
#    extensions.ExponentialShift('lr', 0.1, init=1e-3),
#    trigger=triggers.ManualScheduleTrigger([3,4], 'epoch'))

#trainer.extend(
#    DetectionVOCEvaluator(
#        test_iter, model, use_07_metric=True,
#        label_names=voc_bbox_label_names),
#    trigger=(300, 'iteration'))

log_interval = 100, 'iteration'
trainer.extend(extensions.LogReport(trigger=log_interval))
trainer.extend(extensions.observe_lr(), trigger=log_interval)
trainer.extend(extensions.PrintReport(
    ['epoch', 'iteration', 'lr', 'elapsed_time','main/loss', 'main/loss/loc', 'main/loss/conf', 'validation/main/map']),
    trigger=log_interval)
trainer.extend(extensions.ProgressBar(update_interval=500))

#trainer.extend(extensions.snapshot(), trigger=(10000, 'iteration'))
trainer.extend(
    extensions.snapshot_object(model, 'model_iter_{.updater.iteration}'),
    trigger=(100, 'iteration'))

trainer.run()

Exception in main training loop: __call__() takes 2 positional arguments but 4 were given
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/chainer/training/trainer.py", line 306, in run
    update()
  File "/usr/local/lib/python3.6/dist-packages/chainer/training/updaters/standard_updater.py", line 149, in update
    self.update_core()
  File "/usr/local/lib/python3.6/dist-packages/chainer/training/updaters/standard_updater.py", line 160, in update_core
    optimizer.update(loss_func, *in_arrays)
  File "/usr/local/lib/python3.6/dist-packages/chainer/optimizer.py", line 650, in update
    loss = lossfun(*args, **kwds)
Will finalize trainer extensions and updater before reraising the exception.


TypeError: ignored